In [102]:
from termcolor import colored
import pandas as pd
import numpy as np
import string
import regex as re
import zipfile
import gc
from scipy.stats import boxcox
import sys 
# other python utilities 
from collections import Counter 
from tqdm import tqdm 
import pandas as pd 
import numpy as np 
import warnings, math
from termcolor import colored
import pickle
import string
from sklearn.metrics.pairwise import cosine_similarity
# for eval
from sklearn.model_selection import train_test_split
import random


# for SVD
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
pd.set_option('display.float_format','{:.2f}'.format)

# for TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from sklearn import preprocessing 

In [2]:
df = pd.read_csv("D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Oct_Forth_projectType.csv")

categories_cols = []

for col in df.columns:
    #print(str(col))
    cats = len(df[col].unique())
    #print(cats)
    if cats<=10:
        categories_cols.append(col)
        print(col)
        
df[categories_cols] = df[categories_cols].astype('category')

Project Grade Level Category
Project Resource Category
Project Current Status
School Metro Type
School State
Donor State


In [3]:
df.head(1)

,Project ID,Donation ID,Donor ID,Donation Amount,Donation Received Date,School ID,Project Title,Project Need Statement,Project Subject Category Tree,Project Subject Subcategory Tree,...,Project Cost,Project Posted Date,Project Expiration Date,Project Current Status,Project Fully Funded Date,School Metro Type,School State,School Zip,Donor State,Donor Zip
0,0029e426fd3296af4fc333580fa895fe,061a5652846763d2eca84535978a647b,0e345dcdef0d2a36c9bd17bf1ac3e10a,0.68,2014-08-10 20:32:12,55fe7ff4c71893998d0e9a26a9d3d0b7,"Everyone Needs an Address, Especially Maniac M...",My students need a class set of the book Mania...,"Applied Learning, Literacy & Language","Character Education, Literature & Writing",...,510.00,2014-06-12,2014-10-09,Fully Funded,2014-08-23,suburban,Georgia,30238,California,900.00


In [4]:
# only for checking
donor_project = df.groupby(['Donor ID', 'Project ID'])['Donation Amount'].sum().reset_index()
print(f"{len(donor_project)}")
print(f"{len(df)}")


22919
48546


In [6]:
"""# create a df for 

donors_projects_df = donor_project.pivot('Donor ID', 'Project ID', 'Donation Amount').fillna(0)

donors_projects = donors_projects_df.values
donors_id = donors_projects_df.index
projects_id = donors_projects_df.columns

print(f'length of donors: {len(donors_id)}')
print(f'length of projects: {len(projects_id)}')
"""

In [149]:
!pip install --upgrade scipy --user

In [52]:
len(projects_df)

860

In [54]:

# create a df for projects specifications
projects_df = df[['Project ID', 'Project Subject Subcategory Tree', 'Project Title', 'Project Need Statement', 'School State']].drop_duplicates(['Project ID']).reset_index(drop=True)
projects_id = projects_df['Project ID'].tolist()
print("len(projects_id) = ", len(projects_id) )
# clean the text
patterns = ['Â', '']
punct = re.sub(r'[\&\.\,]', '', string.punctuation)

def clean(text):
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    text = text.translate(str.maketrans('', '', punct))
    text = text.lower()
    text = re.sub(r' +(?![iaIA])[\w] +', ' ', text)
    text = re.sub(r' \s+', ' ', text)
    
    return text

# add them up
project_txt = []

for i in tqdm(range(len(projects_df)), position=0, leave=True):
    project_txt.append(
        clean(projects_df.loc[i, 'Project Title'] + ' & ' + projects_df.loc[i, 'Project Need Statement'] + ' & ' + projects_df.loc[i, 'Project Subject Subcategory Tree'] + ' & ' + projects_df.loc[i, 'School State'])
        )

projects_df.loc[:, 'Project TFIDF Text'] = project_txt
print("len(project_txt) = ", len(project_txt))

100%|██████████| 860/860 [00:00<00:00, 9581.63it/s]

len(projects_id) =  860
len(project_txt) =  860


In [55]:
# TFIDF
vectorizer = TfidfVectorizer(max_df = 0.99,    
                             min_df = 0.001,
                             stop_words='english',
                             strip_accents='unicode',
                             analyzer='word')

tfidf_matrix = vectorizer.fit_transform(project_txt)

print(f"tfidf_matrix.shape = {tfidf_matrix.shape}")

tfidf_matrix.shape = (860, 3074)


In [56]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)

print('# donations in Train set: ', f"{len(df_train):,}")
print('# donations in Test set: ',  f"{len(df_test):,}")

print('# Donors in Train set: ', f"{len(df_train['Donor ID'].unique()):,}")
print('# Donors in Test set: ',  f"{len(df_test['Donor ID'].unique()):,}")


print('# Donors in both Train and Test sets - the ones we choose for evaluation: ',
      colored(f"{len(df_train[df_train['Donor ID'].isin(df_test['Donor ID'].values.tolist())]['Donor ID'].unique()):,}", 'blue'))



df_train = df_train[df_train['Donor ID'].isin(df_test['Donor ID'].values.tolist())].reset_index(drop=True)
df_test = df_test[df_test['Donor ID'].isin(df_train['Donor ID'].values.tolist())].reset_index(drop=True)

# sum of donation in a grouped by donor id dataset
df_train_donor_index = df_train.groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')
df_test_donor_index = df_test.groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')

# donations in Train set:  43,691
# donations in Test set:  4,855
# Donors in Train set:  795
# Donors in Test set:  618
# Donors in both Train and Test sets - the ones we choose for evaluation:  618


In [11]:
test_train_comparison_df = pd.merge(right = df_train.groupby('Donor ID').count()[['Donation ID']].reset_index(), left = df_test.groupby('Donor ID').count()[['Donation ID']].reset_index(), on='Donor ID', suffixes=('_test', '_train')).sort_values(by='Donation ID_test').reset_index(drop=True)
test_train_comparison_df = pd.merge(right = test_train_comparison_df, left = df_train.groupby('Donor ID'). agg({"Project ID": "nunique"})[['Project ID']].reset_index().rename(columns={'Project ID': 'donated_projects_train'}), on='Donor ID')
test_train_comparison_df = pd.merge(right = test_train_comparison_df, left = df_test.groupby('Donor ID'). agg({"Project ID": "nunique"})[['Project ID']].reset_index().rename(columns={'Project ID': 'donated_projects_test'}), on='Donor ID')

test_train_comparison_df.sort_values(by='donated_projects_test').reset_index(drop=True)

,Donor ID,donated_projects_test,donated_projects_train,Donation ID_test,Donation ID_train
0,7f7fca83ead83a4bb01123429d3114ef,1,8,1,9
1,787a3bb8aaf99727563c1f62f98610c5,1,5,1,5
2,774836dd111e337194f91e517df9eb94,1,21,1,35
3,76d0cdb881c521e1743118634f0c5f28,1,8,1,14
4,f0ea78ff6ce64e89e929948a60ea09d9,1,22,2,28
...,...,...,...,...,...
613,c376c98b0cdb746cf025bb21ee810376,49,239,53,410
614,39df9399f5384334a42905bcf0acdcbf,57,57,954,8727
615,4416745560343f14a74dedcda4ec03b0,60,238,68,538
616,b51c76411b51751f45527c63c69ead9e,67,352,77,702


### EXP

In [88]:
test_train_comparison_df[test_train_comparison_df['Donor ID'] == '0a497ef14df378d153a16dc7a1abaf8b']

,Donor ID,donated_projects_test,donated_projects_train,Donation ID_test,Donation ID_train
14,0a497ef14df378d153a16dc7a1abaf8b,1,1,2,29


In [134]:
len(donor_donated_project_intrain_profiles)

21

In [89]:
def get_projects_profiles_tfidf(ids, tfidf_matrix):

    """
    input: a series of project ids
    output: a matrix of projects tfidfs
    """
    # repeat for all projects
    for project_id in np.ravel([ids]):
        print(projects_id.index(project_id))

    profiles_list = [get_project_profile_tfidf(project_id, tfidf_matrix) for project_id in np.ravel([ids])]
    assert len(profiles_list) == len(ids)


    # stack them onto each other onto a list
    project_profiles = scipy.sparse.vstack(profiles_list)
    assert project_profiles.shape[0] == len(ids)

    return profiles_list, project_profiles



donor_id = '774836dd111e337194f91e517df9eb94'
print("donor_id = ", donor_id)
donor_train_donations_series = df_train_donor_index.loc[donor_id]
print(f'donor_train_donations_series: {len(donor_train_donations_series)}')
assert len(donor_train_donations_series['Project ID']) == len(donor_train_donations_series['Project ID'].unique())

# get the vectors of projects this person has donated to
donor_donated_project_intrain_profiles = get_projects_profiles_tfidf(donor_train_donations_series['Project ID'], tfidf_matrix)

# get the smoothed donated amount as the weight of each project
donor_project_strengths = np.array(donor_train_donations_series['Donation Amount']).reshape(-1, 1)


print(f"donor_project_strengths.shape = {donor_project_strengths.shape} ")
#print("donor_donated_project_intrain_profiles = ", donor_donated_project_intrain_profiles.shape)
#print("donor_donated_project_intrain_profiles[0] = ", donor_donated_project_intrain_profiles[0].shape)


# multiply the weights and vectors
#multiplication = donor_donated_project_intrain_profiles.multiply(donor_project_strengths, )
#print('multiplication.shape = ', multiplication.shape)
#print('multi: ', multiplication)
#print("np.sum(multiplication, axis=0) = ", np.sum(multiplication, axis=0))


# now we normalize the whole vector 
#normalized_donor_preference = preprocessing.normalize(multiplication)



donor_id =  774836dd111e337194f91e517df9eb94
donor_train_donations_series: 21
271
5
9
11
709
219
681
40
276
304
305
64
170
238
654
334
389
283
265
267
721
donor_project_strengths.shape = (21, 1) 


In [36]:
profiles_list, project_profiles = donor_donated_project_intrain_profiles

In [58]:
[i.todense() for i in profiles_list]

[matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([], shape=(0, 3074), dtype=float64),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([], shape=(0, 3074), dtype=float64),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([], shape=(0, 3074), dtype=float64),
 matrix([], shape=(0, 3074), dtype=float64),
 matrix([], shape=(0, 3074), dtype=float64),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([[0., 0., 0., ..., 0., 0., 0.]]),
 matrix([], shape=(0, 3074), dtype=float64)]

In [41]:
project_profiles.todense().sum(axis=1)

matrix([[4.44978154],
        [3.25510052],
        [2.41601787],
        [4.63996803],
        [3.62637929],
        [4.71571741],
        [3.60379365],
        [2.25056103],
        [4.63544108],
        [2.97307656],
        [3.92293377],
        [4.16444729],
        [2.92013336],
        [3.86502053],
        [2.80821062]])

In [ ]:
project_profiles.todense().sum(axis=1)

In [59]:

# =======================TFIDF=======================#

# -------------------projects-------------------#

# single project 
def get_project_profile_tfidf(project_id, tfidf_matrix):

    # get the ids
    idx = projects_id.index(project_id)
    # get the word count vector for that document
    project_profile = tfidf_matrix[idx:idx+1]
    return project_profile

# multiple projects profile
def get_projects_profiles_tfidf(ids, tfidf_matrix):

    """
    input: a series of project ids
    output: a matrix of projects tfidfs
    """
    # repeat for all projects
    for project_id in np.ravel([ids]):
        print(projects_id.index(project_id))

    profiles_list = [get_project_profile_tfidf(project_id, tfidf_matrix) for project_id in np.ravel([ids])]
    assert len(profiles_list) == len(ids)


    # stack them onto each other onto a list
    project_profiles = scipy.sparse.vstack(profiles_list)
    assert project_profiles.shape[0] == len(ids)

    return profiles_list, project_profiles



# --------------donors------------------#
def build_donor_profile_tfidf(donor_id, df_train_donor_index, tfidf_matrix=tfidf_matrix):

    # get the id of each person and the projects they
    # donated to

    print("donor_id = ", donor_id)
    donor_train_donations_series = df_train_donor_index.loc[donor_id]
    print(f'donor_train_donations_series: {len(donor_train_donations_series)}')
    assert len(donor_train_donations_series['Project ID']) == len(donor_train_donations_series['Project ID'].unique())

    # get the vectors of projects this person has donated to
    donor_donated_project_intrain_profiles = get_projects_profiles_tfidf(donor_train_donations_series['Project ID'], tfidf_matrix)
    print(f"donor_donated_project_intrain_profiles.shape = {donor_donated_project_intrain_profiles.shape} ")

    # get the smoothed donated amount as the weight of each project
    donor_project_strengths = np.array(donor_train_donations_series['Donation Amount']).reshape(-1, 1)
    print(f"donor_project_strengths.shape = {donor_project_strengths.shape} ")

    # multiply the weights and vectors
    multiplication = np.multiply(donor_donated_project_intrain_profiles, donor_project_strengths)
    print('multiplication.shape = ', multiplication.shape)
    print('multi: ', multiplication)
    print("np.sum(multiplication, axis=0) = ", np.sum(multiplication))


    # now we normalize the whole vector 
    normalized_donor_preference = preprocessing.normalize(np.sum(multiplication, axis=0))


    return normalized_donor_preference



def build_donors_profiles_tfidf(df_test_donor_index, df_train_donor_index, sample_size = 1000):
    
    # now for all donors we build a profile in a dictionary
    donor_profiles = {}
    donors_in_test_set = df_test_donor_index.index.unique().tolist()[:sample_size]

    for donor_id in tqdm(donors_in_test_set[:sample_size], position=0, leave=True):
        donor_profiles[donor_id] = build_donor_profile_tfidf(donor_id, df_train_donor_index)

    return donor_profiles



### Cont


In [95]:

# =======================TFIDF=======================#

# -------------------projects-------------------#

# single project 
def get_project_profile_tfidf(project_id, tfidf_matrix):
    """
    input: a project id
    output: the tfidf of that project
    """
    # get the ids
    idx = projects_id.index(project_id)

    # get the word count vector for that document
    project_profile = tfidf_matrix[idx:idx+1]
    return project_profile




# multiple projects profile
def get_projects_profiles_tfidf(ids, tfidf_matrix):

    """
    input: a series of project ids
    output: a matrix of projects tfidfs
    """
    # repeat for all projects

    profiles_list = [get_project_profile_tfidf(project_id, tfidf_matrix) for project_id in np.ravel([ids])]


    # stack them onto each other onto a list
    project_profiles = scipy.sparse.vstack(profiles_list)

    return project_profiles













# --------------donors------------------#

# one donor
def build_donor_profile_tfidf(donor_id, df_train_donor_index, tfidf_matrix=tfidf_matrix):
    """
    input: id of one donor, training set
    output: a one rowed matrix of donor profile, shape: (1, tfidf_vocab_len)
    """

    donor_train_donations_series = df_train_donor_index.loc[donor_id]


    # get the vectors of projects this person has donated to
    donor_donated_project_intrain_profiles = get_projects_profiles_tfidf(donor_train_donations_series['Project ID'], tfidf_matrix)
    

    # get the smoothed donated amount as the weight of each project
    donor_project_strengths = np.array(donor_train_donations_series['Donation Amount']).reshape(-1, 1)


    # multiply the weights and vectors
    multiplication = donor_donated_project_intrain_profiles.multiply(donor_project_strengths)


    # now we normalize the whole vector 
    normalized_donor_preference = preprocessing.normalize(np.sum(multiplication, axis=0))


    return normalized_donor_preference


# multiple donors
def build_donors_profiles_tfidf(df_test_donor_index, df_train_donor_index, sample_size = 1000):
    
    """
    input: donors in test, the trainset, how many donors
    output: a dictionary of "donor_id": "tfidf_projects"

    """
    # now for all donors we build a profile in a dictionary
    donor_profiles = {}
    donors_in_test_set = df_test_donor_index.index.unique().tolist()[:sample_size]

    for donor_id in tqdm(donors_in_test_set[:sample_size], position=0, leave=True):
        donor_profiles[donor_id] = build_donor_profile_tfidf(donor_id, df_train_donor_index)

    return donor_profiles


In [104]:
#-------------------- Content Based------------------ #


class ContentBasedRecommender:
    MODEL_NAME = 'Content-Based'

    def __init__(self, df, donor_profiles):

        self.df = df
        self.donor_profiles = donor_profiles

    def get_model_name(self):
        return self.MODEL_NAME

    def _get_similar_projects_to_donor_profile(self, donor_id, top_n=1000):

        """
        gets the donor id and calculates the cosine similarity between donor profile
        and projects tfidf
        """
        
        cosine_similarities = cosine_similarity(self.donor_profiles[donor_id], tfidf_matrix)

        # sort them and get the indices
        similar_indices = cosine_similarities.argsort().flatten()[-top_n:]
        
        # get the id of project and the score of it
        similar_projects = sorted([(projects_id[i], cosine_similarities[0, i]) for i in similar_indices], key=lambda x:-x[1])
        
        return similar_projects


    def recommend_projects(self, donor_id, projects_to_ignore=[], top_n=10, verbose=False, df_test = df_test):

        """
        gets the donor id 
        """
        donated_projects_test = df_test_donor_index.loc[donor_id, 'Project ID']
        
        similar_projects = self._get_similar_projects_to_donor_profile(donor_id, top_n)

        # remove projects from ignore list if it is also in the test set
        wanted_in_test = []
        for project_id in projects_to_ignore:
            if project_id in donated_projects_test:
                wanted_in_test.append(project_id)

        if wanted_in_test != []: projects_to_ignore = set(projects_to_ignore).difference(wanted_in_test)

        similar_projects_filtered = [x for x in similar_projects if x[0] not in projects_to_ignore]

        recommendations_df = pd.DataFrame(similar_projects_filtered, columns = ['Project ID', 'recommStrength']).head(top_n)

        recommendations_df = recommendations_df.merge(projects_df, how = 'left')[['recommStrength',  'Project ID', 'Project Title', 'Project Need Statement']]

        return recommendations_df




In [97]:
donor_profiles = build_donors_profiles_tfidf(df_test_donor_index, df_train_donor_index)

100%|██████████| 618/618 [00:02<00:00, 207.65it/s]


In [105]:
content_based_model = ContentBasedRecommender(df, donor_profiles)

In [106]:
print(colored('TFIDF: ', 'green'))
global_metrics, detailed_results_df = model_evaluator.evaluate_model(content_based_model)


print('\nGlobal metrics:\n%s' % global_metrics)
detailed_results_df = detailed_results_df[['_donor_id', 'donated_count', "hits@3_count", 'hits@5_count', 'hits@10_count', 'recall@3','recall@5','recall@10']]
detailed_results_df.head(10)

TFIDF: 
0 donors processed
500 donors processed

Global metrics:
{'modelName': 'Content-Based', 'recall@3': 0.052032982655672445, 'recall@5': 0.06880864373045209, 'recall@10': 0.09297696900767699}


,_donor_id,donated_count,hits@3_count,hits@5_count,hits@10_count,recall@3,recall@5,recall@10
0,237db43817f34988f9d543ca518be4ee,90,0,0,1,0.00,0.00,0.01
1,b51c76411b51751f45527c63c69ead9e,67,0,0,1,0.00,0.00,0.01
2,4416745560343f14a74dedcda4ec03b0,60,0,0,0,0.00,0.00,0.00
3,39df9399f5384334a42905bcf0acdcbf,57,0,0,0,0.00,0.00,0.00
4,c376c98b0cdb746cf025bb21ee810376,49,0,0,0,0.00,0.00,0.00
5,a299db9679f7746a805fbc300362191d,44,1,2,2,0.02,0.05,0.05
6,98c4cd327c417683cd76a2ac19fc6254,43,0,2,2,0.00,0.05,0.05
7,03fa60275eb66e873c30ecb86840df4b,42,0,0,0,0.00,0.00,0.00
8,24ecca49933c30a0beb83090591720c0,41,0,0,1,0.00,0.00,0.02
9,609e28d99b36d35679ae56268e4dddc3,39,0,0,1,0.00,0.00,0.03


In [100]:
def get_projects_donated(donor_id, df_index_donor):
    """ get the donors donations """
    try:
        donated_projects = df_index_donor.loc[donor_id]['Project ID']

        return set(donated_projects if type(donated_projects) == pd.Series else [donated_projects])
    
    except KeyError:
        return []


#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_PROJECTS = 100


class ModelEvaluator:


    def get_not_donated_projects_sample(self, donor_id, sample_size, seed=42, projects_id=projects_id):
        
        """ gets the ones that the donor has not donated to """
        df_index_donor = df.set_index('Donor ID')
        donated_projects = get_projects_donated(donor_id, df_index_donor)
        all_projects = set(projects_id)

        not_donated_projects = [x for x in all_projects if x not in donated_projects]
        not_donated_projects_sample = random.sample(not_donated_projects, sample_size)
        
        return set(not_donated_projects_sample)


    def _verify_hit_top_n(self, project_id, recommended_projects, top_n):
        
        """ 
        This function gets the recommended projects and one project id
        Then we search in the recommended projects and output the 
        index of this id among all top n results 
        and whether or not it is in the top n results 
        """
        try:
            index = next(i for i, c in enumerate(recommended_projects) if c == project_id)
        except:
            index = -1

        hit = int(index in range(0, top_n))

        return hit, index


    def evaluate_model_for_donor(self, model, donor_id, df_test_donor_index=df_test_donor_index):
        """
        evaluates the recommendations recommended to one donor
        """
        # what donor has donated to in the test set
        if type(df_test_donor_index.loc[donor_id, 'Project ID']) == pd.Series:
            donated_projects_test = set(df_test_donor_index.loc[donor_id, 'Project ID'])
        else:
            donated_projects_test = set([df_test_donor_index.loc[donor_id, 'Project ID']])
        #print('here3')
        # how many are they?
        donated_projects_count_test = len(donated_projects_test)

        # what the model recommends to the user
        # ignoring the project they have donated to in the
        # training set

        donor_recs_df = model.recommend_projects(donor_id, projects_to_ignore= get_projects_donated(donor_id, df_train_donor_index), top_n = 100, df_test=df_test)

        
        hits_at_3_count = 0
        hits_at_5_count = 0
        hits_at_10_count = 0

        # for each of the projects in the test set
        # they have donated to

         # first get a sample of the ones he\she has not donated to 
        not_donated_projects_sample =  self.get_not_donated_projects_sample(donor_id, sample_size = EVAL_RANDOM_SAMPLE_NON_INTERACTED_PROJECTS, seed = 42)
            
        for project_id in donated_projects_test:

            # add the project id he\she has donated to 
            # to the recommendation list 
            # type(project_id) : sting
            projects_to_filter_recs = not_donated_projects_sample.union(set([project_id]))

            # varifying the projects that are both a random sample
            # or in the top n recommendations of our model
            valid_recs_df =  donor_recs_df[donor_recs_df['Project ID'].isin(projects_to_filter_recs)]

            valid_recs = valid_recs_df['Project ID'].values


            hit_at_3, index_at_3 = self._verify_hit_top_n(project_id, valid_recs, 3)

            hits_at_3_count += hit_at_3
            hit_at_5, index_at_5 = self._verify_hit_top_n(project_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(project_id, valid_recs, 10)
            hits_at_10_count += hit_at_10
        # ---------------------------- Recall --------------------------#
        recall_at_3 = hits_at_3_count/float(donated_projects_count_test)
        recall_at_5 = hits_at_5_count / float(donated_projects_count_test)
        recall_at_10 = hits_at_10_count / float(donated_projects_count_test)


        donor_metrics = {'_donor_id': donor_id,
                        'hits@3_count':hits_at_3_count, 
                         'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'donated_count': donated_projects_count_test,
                          'recall@3': recall_at_3,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return donor_metrics

    def evaluate_model(self, model):

        metrics = []

        for idx, donor_id in enumerate(list(df_test_donor_index.index.unique().values)):
            #print('here1')
            donor_metrics = self.evaluate_model_for_donor(model, donor_id)
            if idx%500 ==0: print('%d donors processed' % idx)

            metrics.append(donor_metrics)

            detailed_results_df = pd.DataFrame(metrics).sort_values('donated_count',  ascending=False).reset_index(drop=True)
            
        glob_num_donations = float(detailed_results_df['donated_count'].sum())

        global_recall_at_3 = detailed_results_df['hits@3_count'].sum()/ glob_num_donations
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum()/ glob_num_donations
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum()/ glob_num_donations

        global_metrics = {'modelName': model.get_model_name(),
                          'recall@3': global_recall_at_3,
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()

